In [2]:
import fitsio as fio
import numpy as np

In [3]:
clus = fio.FITS('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17_lgt20_desformat_catalog.fit')[-1] # Cluster catalog
shape = fio.FITS('/lsst/troxel/y1a1/lgt20_member_mcal_combined.fits')[-1] # Shape/member catalog
mem = fio.FITS('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17_lgt20_desformat_catalog_members.fit')[-1] # Member catalog

In [4]:
cent_id = clus['ID_CENT'].read()[:,0] # Most probable central ID's
shape_id = shape['ID'].read() # Shape/member ID column
mem_id = mem['ID'].read() # Member ID column

In [5]:
# Mask shape/member catalog to select central galaxies
cent_mask = np.in1d(shape_id, cent_id, assume_unique=False)
cent = shape.read()[cent_mask]
r_mask = cent['R']==0
cent = cent[r_mask]

In [6]:
cent_sort = np.argsort(cent['MEM_MATCH_ID'])
cent = cent[cent_sort]

In [7]:
fio.write('/lsst/troxel/y1a1/lgt20_central_shapes.fits', cent, clobber=True)

In [8]:
# Mask member catalog to select satellite galaxies
sat_mask = np.in1d(mem_id, cent['ID'], assume_unique=False, invert=True)
sat = mem.read()[sat_mask]
sat_cent_mask = np.in1d(sat['MEM_MATCH_ID'], cent['MEM_MATCH_ID'], assume_unique=False)
sat = sat[sat_cent_mask]

In [9]:
sat_sort = np.argsort(sat['MEM_MATCH_ID'])
sat = sat[sat_sort]

In [10]:
fio.write('/lsst/troxel/y1a1/lgt20_satellite_members.fits', sat, clobber=True)

In [11]:
p_mem = sat['P']*sat['PFREE']
sat_p80_mask = p_mem >= 0.8
sat_p80 = sat[sat_p80_mask]

In [12]:
fio.write('/lsst/troxel/y1a1/lgt20_satellite_p80.fits', sat_p80, clobber=True)

In [13]:
cent_p80_mask = np.in1d(cent['MEM_MATCH_ID'], sat_p80['MEM_MATCH_ID'], assume_unique=False)
cent_p80 = cent[cent_p80_mask]

In [14]:
fio.write('/lsst/troxel/y1a1/lgt20_central_p80.fits', cent_p80, clobber=True)

In [15]:
sat_p20_mask = p_mem >= 0.2
sat_p20 = sat[sat_p20_mask]

In [16]:
fio.write('/lsst/troxel/y1a1/lgt20_satellite_p20.fits', sat_p20, clobber=True)

In [17]:
cent_p20_mask = np.in1d(cent['MEM_MATCH_ID'], sat_p20['MEM_MATCH_ID'], assume_unique=False)
cent_p20 = cent[cent_p20_mask]

In [18]:
fio.write('/lsst/troxel/y1a1/lgt20_central_p20.fits', cent_p20, clobber=True)